# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [4]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy_database.csv")

# Display sample data
city_data_df

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Haines Junction,CA,60.7522,-137.5108,39.15,65,100,6.91,overcast clouds
1,1,Mar Del Plata,AR,-38.0023,-57.5575,79.83,84,0,13.80,clear sky
2,2,Butaritari,KI,3.0707,172.7902,80.58,77,24,9.08,few clouds
3,3,Kodiak,US,57.7900,-152.4072,44.33,70,34,12.39,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,42.46,61,75,20.71,broken clouds
...,...,...,...,...,...,...,...,...,...,...
663,663,Sao Felix Do Xingu,BR,-6.6447,-51.9950,91.98,41,79,5.57,broken clouds
664,664,Senges,BR,-24.1128,-49.4636,72.03,76,83,5.88,broken clouds
665,665,Micheweni,TZ,-4.9667,39.8333,75.52,79,100,11.74,overcast clouds
666,666,Najran,SA,17.4924,44.1277,68.65,19,8,2.42,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [7]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [8]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mar Del Plata,AR,-38.0023,-57.5575,79.83,84,0,13.80,clear sky
2,2,Butaritari,KI,3.0707,172.7902,80.58,77,24,9.08,few clouds
5,5,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,65,0,24.16,clear sky
8,8,Poum,NC,-20.2333,164.0167,75.74,73,100,16.44,overcast clouds
9,9,Sao Joao Da Barra,BR,-21.6403,-41.0511,82.49,79,26,22.77,scattered clouds
12,12,Georgetown,MY,5.4112,100.3354,82.89,82,20,0.00,few clouds
14,14,Batticaloa,LK,7.7102,81.6924,78.71,82,100,7.99,moderate rain
17,17,Soller,ES,39.7662,2.7152,76.86,76,100,1.01,overcast clouds
19,19,Dingle,PH,10.9995,122.6711,76.64,92,100,1.59,overcast clouds
21,21,Victoria,HK,22.2855,114.1577,75.33,78,92,5.99,overcast clouds


In [10]:
preferred_cities_df.count()

City                   205
City.1                 205
Country                203
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [14]:
preferred_cities_df.isnull().sum()

City                   0
City.1                 0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mar Del Plata,AR,-38.0023,-57.5575,79.83,84,0,13.80,clear sky
2,2,Butaritari,KI,3.0707,172.7902,80.58,77,24,9.08,few clouds
5,5,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,65,0,24.16,clear sky
8,8,Poum,NC,-20.2333,164.0167,75.74,73,100,16.44,overcast clouds
9,9,Sao Joao Da Barra,BR,-21.6403,-41.0511,82.49,79,26,22.77,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
650,650,Calcutta,IN,22.5697,88.3697,78.75,78,20,5.75,haze
653,653,Ixtapa,MX,20.7000,-105.2000,87.22,70,75,8.05,broken clouds
658,658,Hwange,ZW,-18.3693,26.5019,79.39,32,2,3.00,clear sky
665,665,Micheweni,TZ,-4.9667,39.8333,75.52,79,100,11.74,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [45]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,1,AR,79.83,clear sky,-38.0023,-57.5575,
2,2,KI,80.58,few clouds,3.0707,172.7902,
5,5,BR,84.16,clear sky,-22.9661,-42.0278,
8,8,NC,75.74,overcast clouds,-20.2333,164.0167,
9,9,BR,82.49,scattered clouds,-21.6403,-41.0511,
12,12,MY,82.89,few clouds,5.4112,100.3354,
14,14,LK,78.71,moderate rain,7.7102,81.6924,
17,17,ES,76.86,overcast clouds,39.7662,2.7152,
19,19,PH,76.64,overcast clouds,10.9995,122.6711,
21,21,HK,75.33,overcast clouds,22.2855,114.1577,


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [47]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()

# Display sample data
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,1,AR,79.83,clear sky,-38.0023,-57.5575,
2,2,KI,80.58,few clouds,3.0707,172.7902,
5,5,BR,84.16,clear sky,-22.9661,-42.0278,
8,8,NC,75.74,overcast clouds,-20.2333,164.0167,
9,9,BR,82.49,scattered clouds,-21.6403,-41.0511,
...,...,...,...,...,...,...,...
650,650,IN,78.75,haze,22.5697,88.3697,
653,653,MX,87.22,broken clouds,20.7000,-105.2000,
658,658,ZW,79.39,clear sky,-18.3693,26.5019,
665,665,TZ,75.52,overcast clouds,-4.9667,39.8333,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [48]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [49]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [50]:
# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,1,AR,79.83,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
2,2,KI,80.58,few clouds,3.0707,172.7902,Isles Sunset Lodge
5,5,BR,84.16,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,8,NC,75.74,overcast clouds,-20.2333,164.0167,
9,9,BR,82.49,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
...,...,...,...,...,...,...,...
650,650,IN,78.75,haze,22.5697,88.3697,CAMELLIA GROUP
653,653,MX,87.22,broken clouds,20.7000,-105.2000,hotel la terminal
658,658,ZW,79.39,clear sky,-18.3693,26.5019,Hwange Colliery Guest House
665,665,TZ,75.52,overcast clouds,-4.9667,39.8333,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [51]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,1,AR,79.83,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
2,2,KI,80.58,few clouds,3.0707,172.7902,Isles Sunset Lodge
5,5,BR,84.16,clear sky,-22.9661,-42.0278,Pousada Porto Praia
8,8,NC,75.74,overcast clouds,-20.2333,164.0167,
9,9,BR,82.49,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
...,...,...,...,...,...,...,...
650,650,IN,78.75,haze,22.5697,88.3697,CAMELLIA GROUP
653,653,MX,87.22,broken clouds,20.7000,-105.2000,hotel la terminal
658,658,ZW,79.39,clear sky,-18.3693,26.5019,Hwange Colliery Guest House
665,665,TZ,75.52,overcast clouds,-4.9667,39.8333,


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [52]:
# Set the file name.
output_data_file = ("WeatherPy_Vacation.csv")

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [53]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [56]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Gran Hotel Mar del Plata</dd>\n<dt>City</dt><dd>1</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>clear sky and 79.83 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Isles Sunset Lodge</dd>\n<dt>City</dt><dd>2</dd>\n<dt>Country</dt><dd>KI</dd>\n<dt>Current Weather</dt><dd>few clouds and 80.58 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Porto Praia</dd>\n<dt>City</dt><dd>5</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>clear sky and 84.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>8</dd>\n<dt>Country</dt><dd>NC</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 75.74 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Porto De Canoas</dd>\n<dt>City</dt><dd>9</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 82.49 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>12</dd>\n<dt>Country</dt><

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [55]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,-38.0023,-57.5575
2,3.0707,172.7902
5,-22.9661,-42.0278
8,-20.2333,164.0167
9,-21.6403,-41.0511
12,5.4112,100.3354
14,7.7102,81.6924
17,39.7662,2.7152
19,10.9995,122.6711
21,22.2855,114.1577


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [63]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Create a figure to add the Google map as a layer
fig.add_layer(heat_layer)

# Display the figure containing the map
fig.add_layer(marker_layer)

IndexError: list index out of range

In [64]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

IndexError: list index out of range